Defining the Quic client and its functions.

In [1]:
import os
import socket
import struct
import math
import random

LOCAL_IP = '127.0.0.1'
LOCAL_PORT = 12345
PACKET_SIZE = 1024

class QuicClient:
    def __init__(self):
        self.streams = []

    def generate_random_data(self, size):
        return os.urandom(size)

    def generate_random_file(self, file_size):
        with open("random_file.txt", "wb") as file:
            file.write(self.generate_random_data(file_size))

    def create_streams(self, num_streams):
        for i in range(num_streams):
            stream_id = i + 1
            self.streams.append(stream_id)

    def send_file_over_streams(self, file_size):
        with open("random_file.txt", "rb") as file:
            file_data = file.read()
            num_packets = math.ceil(file_size / PACKET_SIZE)
            for stream_id in self.streams:
                for i in range(num_packets):
                    start = i * PACKET_SIZE
                    end = min(start + PACKET_SIZE, file_size)
                    chunk = file_data[start:end]
                    self.send_data(stream_id, chunk)
                self.send_end_of_stream(stream_id)

    def send_data(self, stream_id, data):
        server_address = (LOCAL_IP, LOCAL_PORT)
        with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as sock:
            message = struct.pack("!I", stream_id) + data
            sock.sendto(message, server_address)

    def send_end_of_stream(self, stream_id):
        server_address = (LOCAL_IP, LOCAL_PORT)
        with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as sock:
            message = struct.pack("!I", stream_id) + b"END"
            sock.sendto(message, server_address)
            
    def send_initial_message(self, num_streams, file_size):
        initial_message = struct.pack("!II", num_streams, file_size)
        server_address = (LOCAL_IP, LOCAL_PORT)
        with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as sock:
            sock.sendto(initial_message, server_address)


# Usage example:
client = QuicClient()
two_mb = 2*1024*1024
one_mb = 1*1024*1024
# Generate random file size between 1 and 2 megabytes
file_size = random.randint(one_mb, two_mb)
client.generate_random_file(file_size)

# Generate random number of streams between 1 and 10
num_streams = random.randint(1, 10)
client.create_streams(num_streams)

# Send initial message with the random number of streams and file size
client.send_initial_message(num_streams, file_size)

# Send the file over streams
client.send_file_over_streams(file_size)

os.remove("random_file.txt")



Creating a Quic Client and sending the file.

A random file is being generated of size 1 - 2 mega bytes.
Amount of streams are being generated (1-10)
The client then sends the file over to the receiver. Such that in each stream the same file is being sent once.

In [2]:
# Generate a random file with a size between 1 and 2 MB
file_size = random.randint(1024 * 1024, 2 * 1024 * 1024)
client = QuicClient()
client.generate_random_file(file_size)

# Generate a random number of streams between 1 and 10
num_streams = random.randint(1, 10)

# Create the specified number of streams
client.create_streams(num_streams)

# Send the generated file over multiple streams
client.send_file_over_streams(file_size)

Clean up

In [3]:
#clean up
os.remove("random_file.txt")